# SQL to DataFrame

### The goal of this notebook is to test connections to an Azure SQL Database.

This notebook will cover connecting to an Azure SQL Database from a python notebook. Although there are many options out there as far as baked-in SQL connections (Deepnote, Datacamp Workspace, etc), I need a solution that can easily be used with a Binder enabled GitHub repository. Moving forward, I want to be able to easily switch between desktop and iOS/iPadOS without having to alter my workflow. Although I can access Deepnote and Workspace from mobile devices, it is not as fluid as utilizing Juno Connect with Binder and the Motion-Analysis repository.

I'll be attempting different methods to see which is the most efficient for my workflow -- here are the options I've researched so far:
- pyodbc
- sqlalchemy

## pyodbc

The following code is direct from Microsoft to run a test query on an Azure SQL Database utilizing <code>pyodbc</code>.

In [ ]:
import pyodbc

In [ ]:
server = 'wasted-arrows.database.windows.net'
database = 'MotionData'
username = 'wa_dba'
password = '{kawroj-tukbi5-zywnYz}'
driver = '{ODBC Driver 17 for SQL Server}'


with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+password) as conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT TOP 3 time AS 'time', capture_date AS 'capture_date', capture_sequence AS 'capture_sequence' FROM Accelerometer ORDER BY time DESC")
        row = cursor.fetchone()
        while row:
            print(str(row[0]) + ' ' + str(row[1]) + ' ' + str(row[2]))
            row = cursor.fetchone()


After testing, I turned the test code into a function that accepts a single argument. The argument it accepts is just a string of the SQL statements I wish to pass to the Database.

In [ ]:
def motionDB(statement):
    with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+password) as conn:
        with conn.cursor() as cursor:
            cursor.execute(statement)
            row = cursor.fetchone()
            while row:
                print(str(row[0]) + ' ' + str(row[1]) + ' ' + str(row[2]))
                row = cursor.fetchone()


In [ ]:
# Return the first 10 entries of capture_sequence 4 from the Barometer table
bartop10 = '''
SELECT TOP 10 *
FROM Barometer
WHERE capture_sequence = 4
'''

In [ ]:
motionDB(bartop10)

As seen by the output, the function and argument work as expected. The next step is to convert the returned info into a Pandas DataFrame.

### Part 2

In this part I'll play around with some answers I found on Stack Overflow <i>(see: <u>[StackOverflow: Read data from pyodbc to pandas](https://stackoverflow.com/questions/39835770/read-data-from-pyodbc-to-pandas)</u></i>)

In [62]:
import pandas as pd

cnxn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=tcp:wasted-arrows.database.windows.net,1433;Database=MotionData;Uid=wa_dba;Pwd={kawroj-tukbi5-zywnYz};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')
sql = '''
SELECT TOP 10 *
FROM Accelerometer
WHERE capture_sequence = 6'''
data = pd.read_sql(sql,cnxn)  # without parameters [non-prepared statement]
data.head()
# with a prepared statement, use list/tuple/dictionary of parameters depending on DB
#data = pd.read_sql(sql=sql, con=cnxn, params=query_params)

C:\Users\Kristopher\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,time,seconds_elapsed,z,y,x,capture_date,capture_sequence
0,1667909365595406600,0.051406,-0.190607,-0.095882,-0.065918,2022-11-08,6
1,1667909365605479400,0.061479,-0.204634,-0.103110,-0.046438,2022-11-08,6
2,1667909365615552500,0.071552,-0.127795,-0.124551,-0.062474,2022-11-08,6
3,1667909365625625600,0.081625,-0.049477,-0.133205,-0.099597,2022-11-08,6
4,1667909365635698400,0.091698,0.059434,-0.130430,-0.083131,2022-11-08,6


Testing out the last block resulted in an error. After a short time digging I discovered I was pointing to ODBC Driver 18 erroneously, the correct driver number is 17. I did also find that SQLAlchemy is the preferred package for Pandas. So with that, I'll move on to testing SQLALchemy.

## SQLAlchemy

The following code snippet was found while researching the error from the last block (incorrect driver, <u><i>[StackOverflow: PYODBC--Data source name not found and no default driver specified](https://stackoverflow.com/questions/46045834/pyodbc-data-source-name-not-found-and-no-default-driver-specified)</u></i>). Initially I ran into the same error with the SQLALchemny code, but that was when I discovered the driver number was incorrect.


In [63]:
from sqlalchemy.engine import URL

connection_string = f'''
Driver={driver};
Server=tcp:{server},1433;
Database={database};
Uid={username};
Pwd={password};
Encrypt=yes;
TrustServerCertificate=no;
Connection Timeout=30;
'''
connection_url = URL.create("mssql+pyodbc", \
                            query={"odbc_connect": connection_string})

from sqlalchemy import create_engine
engine = create_engine(connection_url)

In [64]:
sql = '''
SELECT TOP 100 *
FROM WristMotion
WHERE capture_sequence = (SELECT MIN(DISTINCT(capture_sequence)) FROM Microphone)
ORDER BY seconds_elapsed'''

df = pd.read_sql(sql, engine)

In [65]:
df

,time,seconds_elapsed,rotationRateX,rotationRateY,rotationRateZ,gravityX,gravityY,gravityZ,accelerationX,accelerationY,accelerationZ,quaternionW,quaternionX,quaternionY,quaternionZ,capture_date,capture_sequence
0,1666950651452580600,1.531581,-0.028649,-0.028887,0.027291,0.168710,-0.305658,-0.937075,0.032676,0.051004,0.027255,0.984143,0.155291,0.085714,0.000000,2022-10-28,3
1,1666950651462553900,1.541554,0.004334,-0.018860,0.029478,0.168966,-0.305185,-0.937183,0.027583,0.055307,0.014820,0.984171,0.155041,0.085852,0.000064,2022-10-28,3
2,1666950651472526800,1.551527,0.042355,-0.026153,0.035739,0.169213,-0.305506,-0.937034,0.023841,0.046427,-0.000374,0.984133,0.155202,0.085996,0.000160,2022-10-28,3
3,1666950651482499800,1.561500,0.041126,-0.039087,0.026955,0.169424,-0.306567,-0.936649,0.018137,0.029590,-0.013119,0.984035,0.155749,0.086126,0.000251,2022-10-28,3
4,1666950651492471800,1.571472,0.020924,-0.025142,0.032885,0.169808,-0.307804,-0.936174,0.010444,0.007099,-0.030028,0.983914,0.156389,0.086344,0.000329,2022-10-28,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1666950652399994400,2.478994,-0.036416,-0.031295,0.005112,0.149921,-0.292966,-0.944296,-0.001804,-0.007266,0.021673,0.985975,0.148635,0.075892,-0.000892,2022-10-28,3
96,1666950652409966300,2.488966,-0.002551,-0.013635,-0.000927,0.149692,-0.292295,-0.944540,-0.001026,-0.004275,0.029455,0.986037,0.148286,0.075772,-0.000891,2022-10-28,3
97,1666950652419939300,2.498939,0.013605,-0.011246,-0.003043,0.149497,-0.291870,-0.944702,0.000161,-0.000046,0.030838,0.986078,0.148067,0.075665,-0.000927,2022-10-28,3
98,1666950652429912300,2.508912,0.017040,-0.016889,0.003719,0.149322,-0.291740,-0.944770,0.000336,0.002662,0.023918,0.986096,0.148000,0.075570,-0.000957,2022-10-28,3


In [68]:
def db(statement):
    return pd.read_sql(statement, engine)

In [72]:
accel_top = '''
SELECT TOP 10 *
FROM Accelerometer
ORDER BY seconds_elapsed DESC
'''

df_accel_top = db(accel_top)
df_accel_top

,time,seconds_elapsed,z,y,x,capture_date,capture_sequence
0,1667050922889186300,13882.919186,0.295586,-0.163258,-0.085849,2022-10-29,5
1,1667050922879113500,13882.909114,0.258264,0.002989,-0.058666,2022-10-29,5
2,1667050922869040400,13882.899040,0.279514,0.130501,0.113345,2022-10-29,5
3,1667050922858967600,13882.888968,0.067750,-0.177643,0.105313,2022-10-29,5
4,1667050922848894500,13882.878895,-0.194960,-0.526001,-0.022519,2022-10-29,5
5,1667050922838821400,13882.868821,-0.684914,-0.262150,-0.160601,2022-10-29,5
6,1667050922828748500,13882.858749,-1.454575,0.399044,-0.099595,2022-10-29,5
7,1667050922818674700,13882.848675,-1.463843,0.058211,-0.219399,2022-10-29,5
8,1667050922808601300,13882.838601,-0.060962,-0.735100,-1.221866,2022-10-29,5
9,1667050922798528500,13882.828529,1.828231,-0.551260,-0.964108,2022-10-29,5
